In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003) (3).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,23:42]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')


d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,3,5,7,8,9,10,11,13,15,17], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)

d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def tree_model(x):
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def error(x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 8))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
        print(pp,py, r)
    return emin , emax;

rmin = 10
rmax = 11

for x4 in range(rmin,rmax):
                dif=3.0
                model = basic_model_3(x4)
                epochs = 10000
                batch_size = 128
                #print('Epochs: ', epochs)
                #print('Batch size: ', batch_size)
                keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]



                history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=0, # Change it to 2, if wished to observe execution
                validation_data=(X_test, Y_test),
                callbacks=keras_callbacks)
                
                train_score = model.evaluate(X_train, Y_train, verbose=0)
                valid_score = model.evaluate(X_test, Y_test, verbose=0)
            
                emn , ema = error(50)
                print('errmin,errmax,x4:',emn,ema , x4)
               

C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[9.382198]] [[12.19]] [[0.7481792]]
[[5.626067]] [[3.28]] [[2.0473514]]
[[9.010982]] [[9.46]] [[0.9466724]]
[[5.0237784]] [[2.39]] [[2.950947]]
[[10.341999]] [[9.]] [[1.1685928]]
[[9.480199]] [[11.32]] [[0.821031]]
[[10.022967]] [[8.5]] [[1.204151]]
[[11.172571]] [[11.]] [[1.0173262]]
[[10.170721]] [[13.91]] [[0.70945776]]
[[10.946633]] [[11.43]] [[0.95347774]]
[[12.0639305]] [[13.84]] [[0.8612445]]
[[6.093869]] [[3.6]] [[1.9741677]]
[[11.659371]] [[14.]] [[0.8193959]]
[[10.951593]] [[8.54]] [[1.3215457]]
[[7.4485693]] [[3.83]] [[2.2969782]]
[[12.003673]] [[16.67]] [[0.7014506]]
[[9.805554]] [[9.69]] [[1.0133587]]
[[9.71808]] [[12.]] [[0.7917955]]
[[7.248422]] [[13.]] [[0.5190989]]
[[11.065486]] [[9.23]] [[1.224113]]
[[10.27316]] [[8.]] [[1.3266033]]
[[9.6727705]] [[7.48]] [[1.3404922]]
[[9.152648]] [[13.]] [[0.67831504]]
[[9.006194]] [[12.06]] [[0.722885]]
[[9.532397]] [[7.84]] [[1.2488818]]
[[11.2154255]] [[14.]] [[0.7851409]]
[[10.875375]] [[13.87]] [[0.76659197]]
[[10.2478]] [[8.]

In [ ]:
model = tree_model(10)
batch_size = 32
epochs = 5000
history = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,
                shuffle=True, verbose=0, validation_data=(X_test, Y_test), callbacks=keras_callbacks)
                
train_score = model.evaluate(X_train, Y_train, verbose=0)
valid_score = model.evaluate(X_test, Y_test, verbose=0)
            
emn , ema = error(50)
print('errmin,errmax,x4:',emn,ema , x4)

In [ ]:
model.save('k.h5')

In [ ]:
print(d1)

In [ ]:
?range
